In [3]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [4]:
import pandas as pd
data = pd.read_excel('politeness_2k_data.xlsx')
data = data.fillna('_NA_')

In [5]:
label_names = ["target"]
y_train = data[label_names].values

In [7]:
import numpy as np
data['doc_len'] = data['comment'].apply(lambda words: len(words.split(" ")))
max_seq_len = np.round(data['doc_len'].mean() + data['doc_len'].std()).astype(int)

In [15]:
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize


from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])


## preprocessing starting 
raw_docs_train = data['comment'].tolist()
num_classes = len(label_names)

print("pre-processing train data...")

processed_docs_train = []
for doc in tqdm(raw_docs_train):
    tokens = word_tokenize(doc)
    filtered = [word for word in tokens if word not in stop_words]
    processed_docs_train.append(" ".join(filtered))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/amiangshu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
 16%|█▌        | 326/2066 [00:00<00:00, 3244.43it/s]

pre-processing train data...


100%|██████████| 2066/2066 [00:00<00:00, 4779.31it/s]


In [21]:
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence

MAX_NB_WORDS = 10000
from tensorflow.keras.preprocessing.text import Tokenizer


print("tokenizing input data...")
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(processed_docs_train )  #leaky
word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
word_index = tokenizer.word_index
print("dictionary size: ", len(word_index))

#pad sequences
word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)

tokenizing input data...
dictionary size:  7245


In [25]:
print('loading word embeddings...')
import os, re, csv, math, codecs

embeddings_index = {}
f = codecs.open('Embedding/crawl-300d-2M.vec', encoding='utf-8')

for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('found %s word vectors' % len(embeddings_index))

3894it [00:00, 19477.81it/s]

loading word embeddings...


2000005it [01:48, 18455.55it/s]

found 1999997 word vectors


In [27]:
#embedding matrix

print('preparing embedding matrix...')
embed_dim = 300
words_not_found = []
nb_words = min(MAX_NB_WORDS, len(word_index)+1)
embedding_matrix = np.zeros((nb_words, embed_dim))

for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

preparing embedding matrix...
number of null word embeddings: 1245


In [39]:
import tensorflow as tf
from sklearn.model_selection import KFold
kf = KFold(n_splits=10, shuffle=True, random_state=125)
from tensorflow.keras.callbacks import EarlyStopping
#from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import  precision_score
from sklearn.metrics import  f1_score

In [42]:

#only LSTM
from tensorflow.keras import regularizers

from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf


#max_features =22248
#nb_words=22248
embedding_dim =300
sequence_length = 100

def LSTM_model():
    model =  keras.Sequential()
    #model.add(tf.keras.layers.Embedding(max_features +1, embedding_dim, input_length=sequence_length,\
                                    #embeddings_regularizer = regularizers.l2(0.005))) 
    model.add( keras.layers.Embedding(nb_words,embed_dim,input_length=max_seq_len,weights=[embedding_matrix],trainable=False))
    model.add( keras.layers.Dropout(0.4))

    model.add( keras.layers.LSTM(embedding_dim,dropout=0.2, recurrent_dropout=0.2,return_sequences=True,\
                                                             kernel_regularizer=regularizers.l2(0.005),\
                                                             bias_regularizer=regularizers.l2(0.005)))

    model.add( keras.layers.Flatten())

    model.add( keras.layers.Dense(512, activation='relu',\
                                kernel_regularizer=regularizers.l2(0.001),\
                                bias_regularizer=regularizers.l2(0.001),))
    model.add( keras.layers.Dropout(0.4))

    model.add( keras.layers.Dense(8, activation='relu',\
                                kernel_regularizer=regularizers.l2(0.001),\
                                bias_regularizer=regularizers.l2(0.001),))
    model.add( keras.layers.Dropout(0.4))


    model.add( keras.layers.Dense(1,activation='sigmoid'))
    
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(),optimizer=tf.keras.optimizers.Adam(1e-3),metrics=['acc'])
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [43]:
LSTM_model().summary()


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 57, 300)           2173800   
_________________________________________________________________
dropout_1 (Dropout)          (None, 57, 300)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 57, 300)           721200    
_________________________________________________________________
flatten (Flatten)            (None, 17100)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               8755712   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
__________________________

In [47]:
from tensorflow.keras.utils import plot_model

#plot_model(LSTM_model(), to_file='LSTMmodel.png', show_shapes=True, show_layer_names=True)


In [50]:
es_callback = EarlyStopping(monitor='val_loss', patience=3)


In [53]:

lstm_run_precision = []
lstm_run_recall = []
lstm_run_f1score = []
lstm_run_accuracy = []

In [54]:
count = 1
num_epochs = 40

for train_index, test_index in kf.split(word_seq_train):
    x_trn, x_tst = word_seq_train[train_index], word_seq_train[test_index]
    y_trn, y_tst = y_train[train_index], y_train[test_index]
    
    x_new_train, x_val, y_new_train, y_val= train_test_split(x_trn, y_trn, test_size=0.11115, random_state=125)
    
    print("\nFold ", count)
    lstm_model=LSTM_model()

    
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
    
    history =lstm_model.fit( x_new_train, y_new_train, batch_size=256,
          epochs=num_epochs, validation_data=(x_val, y_val), callbacks=[es_callback], shuffle=False)
    
    _, train_acc =lstm_model.evaluate(x_new_train,  y_new_train, verbose=0)
    _, val_acc =  lstm_model.evaluate(x_val, y_val, verbose=0)
    print('Train: %.3f, Test: %.3f' % (train_acc, val_acc))
    
    
     
    #plt.savefig('LSTM with fasttext SE data accuracy graph.png')
    #plt.show()
    
    
    y_pred = lstm_model.predict(x_tst)
    y_pred = (y_pred >= 0.5)
 
    
    from sklearn import metrics
    print(metrics.classification_report(y_tst, y_pred))
    
    lstm_precision = precision_score(y_tst, y_pred, pos_label=1)
    lstm_recall = recall_score(y_tst, y_pred, pos_label=1)
    lstm_f1score = f1_score(y_tst, y_pred, pos_label=1)
    lstm_accuracy = accuracy_score(y_tst, y_pred)

    lstm_run_accuracy.append(lstm_accuracy)
    lstm_run_f1score.append(lstm_f1score)
    lstm_run_precision.append(lstm_precision)
    lstm_run_recall.append(lstm_recall)
    
    count = count+1


Fold  1
Train on 1652 samples, validate on 207 samples
Epoch 1/40
1652/1652 [==============================] - 1s 902us/sample - loss: 5.1032 - acc: 0.7100 - val_loss: 4.3410 - val_acc: 0.8019
Epoch 2/40
1652/1652 [==============================] - 0s 218us/sample - loss: 4.0463 - acc: 0.8154 - val_loss: 3.5545 - val_acc: 0.8019
Epoch 3/40
1652/1652 [==============================] - 0s 222us/sample - loss: 3.3733 - acc: 0.8154 - val_loss: 3.0659 - val_acc: 0.8019
Epoch 4/40
1652/1652 [==============================] - 0s 221us/sample - loss: 2.8874 - acc: 0.8160 - val_loss: 2.6460 - val_acc: 0.8019
Epoch 5/40
1652/1652 [==============================] - 0s 221us/sample - loss: 2.5572 - acc: 0.8184 - val_loss: 2.3860 - val_acc: 0.8019
Epoch 6/40
1652/1652 [==============================] - 0s 220us/sample - loss: 2.3503 - acc: 0.8172 - val_loss: 2.2243 - val_acc: 0.8068
Epoch 7/40
1652/1652 [==============================] - 0s 216us/sample - loss: 2.1739 - acc: 0.8226 - val_loss: 2.0

Epoch 17/40
1652/1652 [==============================] - 0s 217us/sample - loss: 1.6792 - acc: 0.8438 - val_loss: 1.6899 - val_acc: 0.8357
Epoch 18/40
1652/1652 [==============================] - 0s 209us/sample - loss: 1.6344 - acc: 0.8686 - val_loss: 1.6645 - val_acc: 0.8406
Epoch 19/40
1652/1652 [==============================] - 0s 215us/sample - loss: 1.6160 - acc: 0.8571 - val_loss: 1.6715 - val_acc: 0.8261
Epoch 20/40
1652/1652 [==============================] - 0s 211us/sample - loss: 1.6184 - acc: 0.8499 - val_loss: 1.6356 - val_acc: 0.8357
Epoch 21/40
1652/1652 [==============================] - 0s 211us/sample - loss: 1.6022 - acc: 0.8535 - val_loss: 1.6578 - val_acc: 0.8019
Epoch 22/40
1652/1652 [==============================] - 0s 213us/sample - loss: 1.5719 - acc: 0.8571 - val_loss: 1.6233 - val_acc: 0.8213
Epoch 23/40
1652/1652 [==============================] - 0s 210us/sample - loss: 1.5389 - acc: 0.8656 - val_loss: 1.5995 - val_acc: 0.8454
Epoch 24/40
1652/1652 [====

1652/1652 [==============================] - 0s 219us/sample - loss: 1.2554 - acc: 0.9019 - val_loss: 1.3665 - val_acc: 0.8406
Epoch 40/40
1652/1652 [==============================] - 0s 213us/sample - loss: 1.2126 - acc: 0.9098 - val_loss: 1.3415 - val_acc: 0.8502
Train: 0.960, Test: 0.850
              precision    recall  f1-score   support

           0       0.87      0.98      0.92       171
           1       0.71      0.28      0.40        36

    accuracy                           0.86       207
   macro avg       0.79      0.63      0.66       207
weighted avg       0.84      0.86      0.83       207


Fold  4
Train on 1652 samples, validate on 207 samples
Epoch 1/40
1652/1652 [==============================] - 2s 1ms/sample - loss: 5.0608 - acc: 0.7500 - val_loss: 4.3900 - val_acc: 0.7826
Epoch 2/40
1652/1652 [==============================] - 0s 222us/sample - loss: 4.0273 - acc: 0.8220 - val_loss: 3.6127 - val_acc: 0.7826
Epoch 3/40
1652/1652 [=============================

1653/1653 [==============================] - 0s 215us/sample - loss: 1.8902 - acc: 0.8572 - val_loss: 1.9748 - val_acc: 0.8502
Epoch 11/40
1653/1653 [==============================] - 0s 212us/sample - loss: 1.8446 - acc: 0.8621 - val_loss: 1.9153 - val_acc: 0.8502
Epoch 12/40
1653/1653 [==============================] - 0s 209us/sample - loss: 1.7841 - acc: 0.8766 - val_loss: 1.9448 - val_acc: 0.8406
Epoch 13/40
1653/1653 [==============================] - 0s 215us/sample - loss: 1.7708 - acc: 0.8730 - val_loss: 1.8976 - val_acc: 0.8406
Epoch 14/40
1653/1653 [==============================] - 0s 215us/sample - loss: 1.7428 - acc: 0.8675 - val_loss: 1.8206 - val_acc: 0.8406
Epoch 15/40
1653/1653 [==============================] - 0s 213us/sample - loss: 1.7074 - acc: 0.8724 - val_loss: 1.8153 - val_acc: 0.8116
Epoch 16/40
1653/1653 [==============================] - 0s 214us/sample - loss: 1.6763 - acc: 0.8832 - val_loss: 1.8180 - val_acc: 0.8406
Epoch 17/40
1653/1653 [================

1653/1653 [==============================] - 0s 209us/sample - loss: 1.5411 - acc: 0.8953 - val_loss: 1.7269 - val_acc: 0.8068
Epoch 24/40
1653/1653 [==============================] - 0s 212us/sample - loss: 1.5174 - acc: 0.8820 - val_loss: 1.7276 - val_acc: 0.7585
Epoch 25/40
1653/1653 [==============================] - 0s 214us/sample - loss: 1.4954 - acc: 0.9050 - val_loss: 1.7218 - val_acc: 0.7729
Epoch 26/40
1653/1653 [==============================] - 0s 211us/sample - loss: 1.4614 - acc: 0.9038 - val_loss: 1.6937 - val_acc: 0.8261
Epoch 27/40
1653/1653 [==============================] - 0s 213us/sample - loss: 1.4253 - acc: 0.9165 - val_loss: 1.6463 - val_acc: 0.8309
Epoch 28/40
1653/1653 [==============================] - 0s 212us/sample - loss: 1.4120 - acc: 0.9087 - val_loss: 1.6185 - val_acc: 0.8019
Epoch 29/40
1653/1653 [==============================] - 0s 213us/sample - loss: 1.3912 - acc: 0.9177 - val_loss: 1.6133 - val_acc: 0.7971
Epoch 30/40
1653/1653 [================

In [ ]:
## for bilstm model

In [70]:
import tensorflow
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Dense, Input, LSTM,GlobalMaxPool1D
maxlen=max_seq_len
embed_size=300
max_features=nb_words
def Bi_LSTM_base():
    inp = keras.layers.Input(shape=(maxlen,))
    x = tensorflow.keras.layers.Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = tensorflow.keras.layers.Bidirectional(tensorflow.keras.layers.LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
    x = tensorflow.keras.layers.GlobalMaxPool1D()(x)
    x = tensorflow.keras.layers.Dense(50, activation="relu")(x)
    x = tensorflow.keras.layers.Dropout(0.1)(x)
    x = tensorflow.keras.layers.Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [71]:
Bi_LSTM_base().summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 57)]              0         
_________________________________________________________________
embedding_18 (Embedding)     (None, 57, 300)           2173800   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 57, 100)           140400    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)               0         
_________________________________________________________________
dense_50 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_50 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_51 (Dense)             (None, 1)                 51    

In [72]:

blbase_run_precision = []
blbase_run_recall = []
blbase_run_f1score = []
blbase_run_accuracy = []

In [73]:
count = 1

for train_index, test_index in kf.split(word_seq_train):
    x_trn, x_tst = word_seq_train[train_index], word_seq_train[test_index]
    y_trn, y_tst = y_train[train_index], y_train[test_index]
    
    x_new_train, x_val, y_new_train, y_val= train_test_split(x_trn, y_trn, test_size=0.11115, random_state=125)
    
    print("\nFold ", count)
    bilstmbase_model=Bi_LSTM_base()

    #model_lstm_fasttext=model_with_embedding()
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
    
    history =bilstmbase_model.fit( x_new_train, y_new_train, batch_size=32,
          epochs=num_epochs, validation_data=(x_val, y_val), callbacks=[es_callback], shuffle=False)
    
    _, train_acc = bilstmbase_model.evaluate(x_new_train,  y_new_train, verbose=0)
    _, val_acc =  bilstmbase_model.evaluate(x_val, y_val, verbose=0)
    print('Train: %.3f, Test: %.3f' % (train_acc, val_acc))
    
    
    y_pred = bilstmbase_model.predict(x_tst)
    y_pred = (y_pred >= 0.5)
 
    
    from sklearn import metrics
    print(metrics.classification_report(y_tst, y_pred))
    
    blbase_precision = precision_score(y_tst, y_pred, pos_label=1)
    blbase_recall = recall_score(y_tst, y_pred, pos_label=1)
    blbase_f1score = f1_score(y_tst, y_pred, pos_label=1)
    blbase_accuracy = accuracy_score(y_tst, y_pred)

    blbase_run_accuracy.append(blbase_accuracy)
    blbase_run_f1score.append(blbase_f1score)
    blbase_run_precision.append(blbase_precision)
    blbase_run_recall.append(blbase_recall)
    
    count = count+1


Fold  1
Train on 1652 samples, validate on 207 samples
Epoch 1/40
1652/1652 [==============================] - 8s 5ms/sample - loss: 0.5059 - acc: 0.8063 - val_loss: 0.4537 - val_acc: 0.8019
Epoch 2/40
1652/1652 [==============================] - 5s 3ms/sample - loss: 0.3583 - acc: 0.8311 - val_loss: 0.3455 - val_acc: 0.8599
Epoch 3/40
1652/1652 [==============================] - 5s 3ms/sample - loss: 0.1632 - acc: 0.9401 - val_loss: 0.3852 - val_acc: 0.8502
Epoch 4/40
1652/1652 [==============================] - 5s 3ms/sample - loss: 0.0393 - acc: 0.9897 - val_loss: 0.6935 - val_acc: 0.8647
Epoch 5/40
1652/1652 [==============================] - 5s 3ms/sample - loss: 0.0114 - acc: 0.9988 - val_loss: 0.7933 - val_acc: 0.8551
Train: 0.999, Test: 0.855
              precision    recall  f1-score   support

           0       0.86      0.96      0.91       165
           1       0.73      0.38      0.50        42

    accuracy                           0.85       207
   macro avg       0

Epoch 1/40
1653/1653 [==============================] - 8s 5ms/sample - loss: 0.4898 - acc: 0.8155 - val_loss: 0.4380 - val_acc: 0.7971
Epoch 2/40
1653/1653 [==============================] - 5s 3ms/sample - loss: 0.3423 - acc: 0.8391 - val_loss: 0.3562 - val_acc: 0.8454
Epoch 3/40
1653/1653 [==============================] - 5s 3ms/sample - loss: 0.1563 - acc: 0.9480 - val_loss: 0.4056 - val_acc: 0.8454
Epoch 4/40
1653/1653 [==============================] - 5s 3ms/sample - loss: 0.0593 - acc: 0.9849 - val_loss: 0.6372 - val_acc: 0.8599
Epoch 5/40
1653/1653 [==============================] - 5s 3ms/sample - loss: 0.0183 - acc: 0.9964 - val_loss: 0.6537 - val_acc: 0.8406
Train: 1.000, Test: 0.841
              precision    recall  f1-score   support

           0       0.85      0.92      0.88       165
           1       0.52      0.37      0.43        41

    accuracy                           0.81       206
   macro avg       0.69      0.64      0.66       206
weighted avg       0.7